In [7]:
import urllib.request
import re

In [2]:
url = ("https://raw.githubusercontent.com/rasbt/"
 "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
 "the-verdict.txt")

In [3]:
file_path = "the-verdict.txt"
urllib.request.urlretrieve(url, file_path)

('the-verdict.txt', <http.client.HTTPMessage at 0x17862f41c70>)

In [6]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
 raw_text = f.read()
print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [10]:
text = "Hello, world. This, is a test."
result = re.split(r'(\s)',text)
result

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']

In [11]:
result = re.split(r'([,.]|\s)', text)
result

['Hello',
 ',',
 '',
 ' ',
 'world',
 '.',
 '',
 ' ',
 'This',
 ',',
 '',
 ' ',
 'is',
 ' ',
 'a',
 ' ',
 'test',
 '.',
 '']

In [12]:
result = [ item for item in result if item.strip()]
result

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']

In [13]:
text = "Hello, world. Is this-- a test?" # Input Text
result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
result

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']

### let’s apply it to Edith Wharton’s entire short story:

In [14]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
len(preprocessed)

4690

In [19]:
raw_text[0:150] # Input text

'I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no great surprise to me to hear that, in the height of'

In [20]:
preprocessed[0:20] # Tokenized text

['I',
 'HAD',
 'always',
 'thought',
 'Jack',
 'Gisburn',
 'rather',
 'a',
 'cheap',
 'genius',
 '--',
 'though',
 'a',
 'good',
 'fellow',
 'enough',
 '--',
 'so',
 'it',
 'was']

In [21]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)
vocab_size

1130

In [23]:
vocab = {token:integer for integer,token in enumerate(all_words)}
for i, item in enumerate(vocab.items()):
 print(item)
 if i >= 50:
     break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


### Implementing a simple text tokenizer

In [35]:
class SimpleTokenizerV1:
 def __init__(self, vocab):
     self.str_to_int = vocab
     self.int_to_str = {i:s for s,i in vocab.items()}

 def encode(self, text):
     preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
     preprocessed = [
     item.strip() for item in preprocessed if item.strip()
     ]
     ids = [self.str_to_int[s] for s in preprocessed]
     return ids

 def decode(self, ids):
     text = " ".join([self.int_to_str[i] for i in ids])

     text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
     return text

In [36]:
tokenizer = SimpleTokenizerV1(vocab)
text = """"It's the last he painted, you know,"
 Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
ids # encode

[1,
 56,
 2,
 850,
 988,
 602,
 533,
 746,
 5,
 1126,
 596,
 5,
 1,
 67,
 7,
 38,
 851,
 1108,
 754,
 793,
 7]

In [37]:
tokenizer.decode(ids) #decode

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [39]:
text = "Hello, do you like tea?"
tokenizer.encode(text)

KeyError: 'Hello'

In [1]:
!pip install tiktoken

  Using cached tiktoken-0.8.0-cp312-cp312-win_amd64.whl.metadata (6.8 kB)
  Using cached regex-2024.9.11-cp312-cp312-win_amd64.whl.metadata (41 kB)
Using cached tiktoken-0.8.0-cp312-cp312-win_amd64.whl (883 kB)
Using cached regex-2024.9.11-cp312-cp312-win_amd64.whl (273 kB)


In [2]:
from importlib.metadata import version
import tiktoken
print("tiktoken version:", version("tiktoken"))

tiktoken version: 0.8.0


In [3]:
tokenizer = tiktoken.get_encoding('gpt2')

In [4]:
text = (
 "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
 "of someunknownPlace."
)
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [5]:
strings = tokenizer.decode(integers)
print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.
